In [1]:
%%capture
%cd '../../src'

import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '../../Datasets/data/new_binance/BTC_futures_1m.csv'

In [3]:
df = pd.read_csv(path, index_col = 0)
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.drop(columns = ['tic'], inplace = True)
df.set_index('Datetime', inplace = True)
df

,Open,High,Low,Close,Volume
Datetime,,,,,
2020-01-01 00:00:00,7189.43,7190.52,7177.00,7182.44,246.092
2020-01-01 00:01:00,7182.43,7182.44,7178.75,7179.01,70.909
2020-01-01 00:02:00,7179.01,7179.01,7175.25,7177.93,99.420
2020-01-01 00:03:00,7177.77,7182.60,7177.00,7181.11,69.330
2020-01-01 00:04:00,7179.10,7179.10,7172.94,7175.25,97.368
...,...,...,...,...,...
2023-10-13 22:55:00,26794.20,26798.40,26755.00,26757.90,1476.125
2023-10-13 22:56:00,26757.90,26787.30,26737.10,26777.30,1091.264
2023-10-13 22:57:00,26777.30,26802.80,26777.20,26800.10,353.109


In [4]:
## sort by datetime
span0 = 180
df0 = df.index.searchsorted(df.index - pd.Timedelta(minutes = 1))
df0 = df0[df0 > 0]
df0=pd.Series(df.index[df0 - 1], index=df.index[df.shape[0]-df0.shape[0]:])
df0=df.loc[df0.index]/df.loc[df0.values].values-1 
# daily returns
df0=df0.ewm(span=span0).std()

In [5]:
df0

,Open,High,Low,Close,Volume
Datetime,,,,,
2020-01-01 00:02:00,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:03:00,0.000566,0.001148,4.203096e-08,0.000651,0.405693
2020-01-01 00:04:00,0.000732,0.000932,4.523642e-05,0.000475,0.330796
2020-01-01 00:05:00,0.000610,0.000768,1.301015e-04,0.000469,0.309368
2020-01-01 00:06:00,0.000531,0.000676,3.373713e-04,0.000492,0.299706
...,...,...,...,...,...
2023-10-13 22:55:00,0.001043,0.001128,1.065089e-03,0.001063,30.358106
2023-10-13 22:56:00,0.001063,0.001128,1.079261e-03,0.001059,30.189939
2023-10-13 22:57:00,0.001058,0.001123,1.077932e-03,0.001068,30.024459


In [6]:
# Average True Range (a technical analysis indicator that measures market volatility)
win = 10    # 5 minutes
atr_df = pd.Series(index = df.index)
high = pd.Series(df['High'].rolling(win, min_periods = win))
low = pd.Series(df['Low'].rolling(win, min_periods = win))
close = pd.Series(df['Close'].rolling(win, min_periods = win))
for i in range(len(df.index)):
    tr = np.max([(high[i] - low[i]), np.abs(high[i] - close[i]), (np.abs(low[i] - close[i]))], axis = 0)
    atr_df[i] = tr.sum() / win
atr_df

Datetime
2020-01-01 00:00:00     1.352
2020-01-01 00:01:00     1.721
2020-01-01 00:02:00     2.097
2020-01-01 00:03:00     2.657
2020-01-01 00:04:00     3.273
                        ...  
2023-10-13 22:55:00    30.850
2023-10-13 22:56:00    33.370
2023-10-13 22:57:00    32.610
2023-10-13 22:58:00    33.510
2023-10-13 22:59:00    31.840
Length: 1990020, dtype: float64

In [7]:
# Tripple barrier
t_final = 180
upper_lower_multipliers = [2, 2]
df1 = df.loc[df0.index]
df1.shape

(1990018, 5)

In [9]:
df1.merge(df0['Close'])

,Open,High,Low,Close,Volume


In [ ]:
df1['top_barrier'] = df1['Close'] + df1['Close'] * upper_lower_multipliers[0] * vol

In [ ]:
barriers = pd.DataFrame(columns = ['minutes_passed', 'price', 'vert_barrier', 'top_barrier', 'bottom_barrier'],
                        index = df0.index)
for ts, row in df0.iterrows():
    vol = row['Close']
    ts_passed = len(df0.loc[df0.index[0] : ts])
    
    # vertical barrier
    if (ts_passed + t_final) < len(df0.index) and (t_final != 0):
        vert_barrier = df0.index[ts_passed + t_final]
    else:
        vert_barrier = np.nan
        
    # top barrier
    if upper_lower_multipliers[0] > 0:
        top_barrier = df1['Close'].loc[ts] + df1['Close'].loc[ts] * upper_lower_multipliers[0] * vol
    else:
        top_barrier = pd.Series(index = df1.index)
    
    # bottom barrier
    if upper_lower_multipliers[1] > 0:
        bottom_barrier = df1['Close'].loc[ts] - df1['Close'].loc[ts] * upper_lower_multipliers[1] * vol
    else:
        bottom_barrier = pd.Series(index = df1.index)
    
    barriers.loc[ts, ['minutes_passed', 'price', 'vert_barrier', 'top_barrier', 'bottom_barrier']] = \
        ts_passed, df1['Close'].loc[ts], vert_barrier, top_barrier, bottom_barrier
        
barriers

In [ ]:
df1